### Simple MLP

Testing new mnist on every epoch


In [1]:
# A Multilayer Perceptron implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)

# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import os
from subprocess import Popen
INFIMNIST = '/Users/charlesmartin14/packages/infimnist/infimnist -d /Users/charlesmartin14/packages/infimnist/data '

class InfiMNIST(object):

    def __init__(self):
        self.data_dir = "."
        self._infimnist_start = 10000
        self._infimnist_stop =  self._infimnist_start + 59999

    def next_epoch(self):
        print "creating infimnist pat files %d - %d" % (self._infimnist_start, self._infimnist_stop)
        lab_file = os.path.join(self.data_dir, 'infimnist-labels')
        pat_file = os.path.join(self.data_dir, 'infimnist-images')
        
        # execute cmd 
        
        with open(lab_file, 'wb') as out:
            cmd = "{} lab {} {} ".format(INFIMNIST, self._infimnist_start, self._infimnist_stop)
            print cmd
            Popen(cmd, shell=True, stdout=out, cwd=self.data_dir).wait()
            
        with open(pat_file, 'wb') as out:
            cmd = "{} pat {} {} ".format(INFIMNIST, self._infimnist_start, self._infimnist_stop)
            print cmd
            Popen(cmd, shell=True, stdout=out, cwd=self.data_dir).wait()
        
        cmd1 = "rm infimnist-labels.gz infimnist-images.gz"
        cmd2 = "gzip -f infimnist-labels infimnist-images"
        cmd3 = "ln -s infimnist-labels.gz  mnist-labels.gz"
        cmd4 = "ln -s infimnist-images.gz  mnist-images.gz"

        os.system(cmd1)
        os.system(cmd2)
        os.system(cmd3)
        os.system(cmd4)
        
        self._infimnist_start = self._infimnist_stop + 1
        self._infimnist_stop =  self._infimnist_start + 59999


In [3]:
infiminst = InfiMNIST()

In [4]:
infiminst.next_epoch()

creating infimnist pat files 10000 - 69999
/Users/charlesmartin14/packages/infimnist/infimnist -d /Users/charlesmartin14/packages/infimnist/data  lab 10000 69999 
/Users/charlesmartin14/packages/infimnist/infimnist -d /Users/charlesmartin14/packages/infimnist/data  pat 10000 69999 


In [5]:
!ls -l *gz

-rw-r--r--+ 1 charlesmartin14  staff  9822045 Apr 22 20:58 infimnist-images.gz
-rw-r--r--+ 1 charlesmartin14  staff    28895 Apr 22 20:58 infimnist-labels.gz
lrwxr-xr-x  1 charlesmartin14  staff       19 Apr 22 20:53 mnist-images.gz -> infimnist-images.gz
lrwxr-xr-x  1 charlesmartin14  staff       19 Apr 22 20:53 mnist-labels.gz -> infimnist-labels.gz
-rw-r--r--+ 1 charlesmartin14  staff  1634324 Apr 22 13:56 t10k-images.gz
-rw-r--r--+ 1 charlesmartin14  staff     4531 Apr 22 13:56 t10k-labels.gz


In [6]:
import input_data

# why does this fail?

In [7]:
mnist = input_data.read_data_sets(".", one_hot=True)

Extracting ./mnist-images.gz
Extracting ./mnist-labels.gz
Extracting ./t10k-images.gz
Extracting ./t10k-labels.gz


/Users/charlesmartin14/anaconda2/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 10000
batch_size = 100
display_step = 25

In [ ]:
# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 256 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [ ]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
# Create model
def multilayer_perceptron(_X, _weights, _biases):
    #Hidden layer with RELU activation
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    #Hidden layer with RELU activation
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.1))
}

In [ ]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [ ]:
# Define loss and optimizer
# Softmax loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
# Adam Optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost) 

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
import input_data

accs = []
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        infiminst.next_epoch()
        mnist = input_data.read_data_sets(".", one_hot=True)
        avg_cost = 0.0
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            a = accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
            accs.append(a)
            print "Accuracy:", a

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

In [ ]:
accs

In [ ]:
infiminst.next_epoch()

In [ ]:
!ls
